In [2]:
from math import log10, log2
# TF
def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0
    
def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1 + log10(freq)
    else:
        return 0
    
def doubleNormalTF(K, freq, maxFreq):
    return K + ((1-K) * (freq/ maxFreq))

In [3]:
collection = [
    ("Document1", "This is a sample"),
    ("Document2", "This is another sample"),
    ("Document3", "This is not sample"),
    ("Document4", "a not sample"),
    ("Document5", "not"),
    ("Document6", "not sample")
]

globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    docIdx = len(globalDocument) # Crawling - Duplicated X
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for token in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if token not in localPosting.keys():
            localPosting[token] =1
        # 있으면, 빈도 증가
        else:
            localPosting[token] += 1
            
    maxFreq = max(localPosting.values())
            
    # Global Merge
    for token, freq in localPosting.items(): 

        if token not in globalLexicon.keys(): #         if 색인어 목록 X -> 새로운 단어 추가
            
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, freq, maxFreq), -1] # -1은 마지막 위치라는 것을 가리키기 위해
            globalPosting.append(postingData)
            globalLexicon[token] = postingIdx
            # 글로발(사전)에서 위치만 가져온, 나중에 posting 위치를 추가
        else: #         else 기존단어의 idx 가져오기
            lexiconIdx = list(globalLexicon.keys()).index(token)
            postingIdx = len(globalPosting)
            oldPostingIdx = globalLexicon[token]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, freq, maxFreq), oldPostingIdx]
            globalPosting.append(postingData)
            globalLexicon[token] = postingIdx

    print(docName)

Document1
Document2
Document3
Document4
Document5
Document6


In [4]:
globalPosting

[[0, 0, 1.0, -1],
 [1, 0, 1.0, -1],
 [2, 0, 1.0, -1],
 [3, 0, 1.0, -1],
 [0, 1, 1.0, 0],
 [1, 1, 1.0, 1],
 [4, 1, 1.0, -1],
 [3, 1, 1.0, 3],
 [0, 2, 1.0, 4],
 [1, 2, 1.0, 5],
 [5, 2, 1.0, -1],
 [3, 2, 1.0, 7],
 [2, 3, 1.0, 2],
 [5, 3, 1.0, 10],
 [3, 3, 1.0, 11],
 [5, 4, 1.0, 13],
 [5, 5, 1.0, 15],
 [3, 5, 1.0, 14]]

In [5]:
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothingIDF(N, df):
    return log10((1+N)/df)  #  < 0 (N=df) 이 경우 -값이 나온다

def probabilityIDF(N, df):
    return log10((N-df+1)/df) #  절반보단 많이 나오면 - 절반보다 적개나오면...+

In [23]:
N = len(globalDocument)
idfList = dict()
documentLength = dict()

for term, ptr in globalLexicon.items():
    
    df = 0
    oldPtr = ptr
    
    print(term)
    
    while True:
        if ptr == -1:
            break
            
        df += 1
        
        postingData = globalPosting[ptr]
        ptr = postingData[3]
        
    ptr = oldPtr
    idf = basicIDF(N, df)
    idfList[term] = idf
    
    print("단어:{0} / IDF:{1}".format(term, idf))

    while True:
        if ptr == -1:
            break
                  
        postingData = globalPosting[ptr]
        tf = postingData[2]
        postingData[2] = tf * idf
        ptr = postingData[3]
        
        print("문서:{0} / TF:{1} / TF-IDF:{2}".format(
            globalDocument[postingData[1]],
            tf,
            postingData[2]
        ))        
        # { 색인어Idx, 문서Idx, TF, Next}
        
        if postingData[1] in documentLength.keys():
            documentLength[postingData[1]] += postingData[2] ** 2
        else:
            documentLength[postingData[1]] = postingData[2] ** 2
    
    idf = basicIDF(N, df)  

this
단어:this / IDF:0.3010299956639812
문서:Document3 / TF:0.09061905828945654 / TF-IDF:0.02727905472394916
문서:Document2 / TF:0.3010299956639812 / TF-IDF:0.09061905828945654
문서:Document1 / TF:0.3010299956639812 / TF-IDF:0.09061905828945654
is
단어:is / IDF:0.3010299956639812
문서:Document3 / TF:0.3010299956639812 / TF-IDF:0.09061905828945654
문서:Document2 / TF:0.3010299956639812 / TF-IDF:0.09061905828945654
문서:Document1 / TF:0.3010299956639812 / TF-IDF:0.09061905828945654
a
단어:a / IDF:0.47712125471966244
문서:Document4 / TF:0.47712125471966244 / TF-IDF:0.227644691705265
문서:Document1 / TF:0.47712125471966244 / TF-IDF:0.227644691705265
sample
단어:sample / IDF:0.07918124604762482
문서:Document6 / TF:0.07918124604762482 / TF-IDF:0.006269669725654501
문서:Document4 / TF:0.07918124604762482 / TF-IDF:0.006269669725654501
문서:Document3 / TF:0.07918124604762482 / TF-IDF:0.006269669725654501
문서:Document2 / TF:0.07918124604762482 / TF-IDF:0.006269669725654501
문서:Document1 / TF:0.07918124604762482 / TF-IDF:0.0062

In [24]:
def euclidean(x, y):
    return (x-y)**2

In [25]:
query = "not"

queryPosting = dict()

localPosting = dict()
    
for token in query.lower().split():
    # Local에 대해서, 없으면 추가
    if token not in queryPosting.keys():
        queryPosting[token] =1
    # 있으면, 빈도 증가
    else:
        queryPosting[token] += 1

maxFreq = max(queryPosting.values())

# Global Merge
for token, freq in queryPosting.items(): 
    queryPosting[token] =  doubleNormalTF(0.5, freq, maxFreq)  * idfList[token]  # [0.5 -1] 쿼리인경우는 K 값을,  문서는

In [26]:
queryPosting

{'not': 0.17609125905568124}

In [36]:
candidateList = dict()

for term, ptr in globalLexicon.items():
    queryWeight = 0
    
    if term in queryPosting.keys():
        queryWeight = queryPosting[term]
        
    while True:
        if ptr == -1:
            break

        postingData = globalPosting[ptr]
        ptr = postingData[3]
        
        if postingData[1] in candidateList.keys():
            candidateList[postingData[1]] += euclidean(queryWeight, postingData[2])
        else:
            candidateList[postingData[1]] = euclidean(queryWeight, postingData[2])

In [37]:
resultList = sorted(candidateList.items(), key=lambda x:x[1])

print("Query")

for i, (fileIdx, distance) in enumerate(resultList):
    print("순위:{0} / 문서:{1} / 거리:{2}".format(
        (i+1), globalDocument[fileIdx], distance
    ))
    print(collection[fileIdx][1])

Query
순위:1 / 문서:Document5 / 거리:0.02104911389674908
not
순위:2 / 문서:Document6 / 거리:0.02108842265521787
not sample
순위:3 / 문서:Document3 / 거리:0.030044383207118004
This is not sample
순위:4 / 문서:Document1 / 거리:0.06828504187058979
This is a sample
순위:5 / 문서:Document4 / 거리:0.07291052831680302
a not sample
순위:6 / 문서:Document2 / 거리:0.38311664180570604
This is another sample


In [38]:
documentLength

{2: 0.009956773530471006,
 1: 0.38311664180570604,
 0: 0.06828504187058979,
 3: 0.05282291864015602,
 5: 0.0010008129785708704,
 4: 0.0009615042201020818}

In [39]:
def innerProduct(x, y):
    return x*y

In [40]:
candidateList = dict()

# for term, ptr in globalLexicon.items():
for term, weight in queryPosting.items():
    
    ptr = globalLexicon[term]

    while True:
        if ptr == -1:
            break

        postingData = globalPosting[ptr]
        ptr = postingData[3]
        
        if postingData[1] in candidateList.keys():
            candidateList[postingData[1]] += innerProduct(weight, postingData[2])
        else:
            candidateList[postingData[1]] = innerProduct(weight, postingData[2])

In [41]:
candidateList

{5: 0.00546026091958402,
 4: 0.00546026091958402,
 3: 0.00546026091958402,
 2: 0.00546026091958402}

In [42]:
for fileIdx, sumProduct in candidateList.items():
    candidateList[fileIdx] = sumProduct / documentLength[fileIdx]

In [44]:
resultList = sorted(candidateList.items(), key= lambda x:x[1], reverse=True)

print("Query: ", query)

for i, (fileIdx, distance) in enumerate(resultList):
    print("순위:{0} / 문서:{1} / 거리:{2}".format(
        (i+1), globalDocument[fileIdx], distance
    ))
    print(collection[fileIdx][1])

Query
순위:1 / 문서:Document5 / 거리:5.6788735872675735
not
순위:2 / 문서:Document6 / 거리:5.45582545040643
not sample
순위:3 / 문서:Document3 / 거리:0.5483966169235268
This is not sample
순위:4 / 문서:Document4 / 거리:0.10336916361590678
a not sample


In [45]:
from konlpy.corpus import kobill
from konlpy.tag import Kkma

ma = Kkma().morphs

def getUniqueTermsBySet():
    lexicon = list() # Unique term List

    # 사전(Lexicon) 만들기 → 우리(Collection = Crawled data) 세상(N차원)
    for fileName in kobill.fileids():
        document = kobill.open(fileName).read()

        for token in document.split():
            lexicon.extend(ma(token))
            
    return list(set(lexicon))

In [46]:
lexicon = getUniqueTermsBySet()

In [47]:
len(lexicon)

1487

In [51]:
from collections import defaultdict

def docRepresentationByDefaultDict():
    docList = defaultdict(lambda: defaultdict(int))
    
#     defaultdict{
#         문서번호(제목):{
#             단어:1
#         }
        
#     }
#     collections = list()


    # 사전(Lexicon) 만들기 → 우리(Collection = Crawled data) 세상(N차원)
    for fileName in kobill.fileids():
#         collections.append(document)
#         collections.index(document)
        document = kobill.open(fileName).read()
#         docVector = list(0 for _ in range(len(lexicon))) # 2638 차원을 갖는 백터
#         docVector = defaultdict(int) # 공간을 차지해서..dict로 변경

        for token in document.split():
            for morpheme in ma(token):
                docList[fileName][morpheme] += 1  # 속도 개선을 위해  index 체크하지 않음
            
    return docList

In [52]:
DTM = docRepresentationByDefaultDict()

In [53]:
# invertedDocument(역문헌구조, 어휘)

# DTM → TDM
#    d1, d2, d3 ...... d10
# t1  1,  0,   1           → Sparse → Dict
# t2
# t3

# Numpy의 transpose를 쓰지 않는 이유 : 데이터 량 (1000개, 2000개 아님  몇십만개, 몇천만개)

def invertedDocument(DTM):
    TDM = defaultdict(lambda: defaultdict(float))
    
    # Only Python → Dictionary | Posting DB
    # Dictionary → term, fp (Hash in memory)
    # Posting → struct(docid, freq, next=fp) (FileDB)
    for docName, docVector in DTM.items():
        maxFreq = max(docVector.values())
        for term, freq in docVector.items():
            TDM[term][docName] = doubleNormalTF(0, freq, maxFreq)
            
    return TDM

In [54]:
TDM = invertedDocument(DTM)

In [63]:
N = len(DTM)

def invertedWeight(TDM):
    TWM = defaultdict(lambda:defaultdict(float))
    # Document Vector Length
    DVL = defaultdict(float)
#     단어:{문서:TF, ....}
    for term, tfList in TDM.items():
        df = len(tfList)
        idf = basicIDF(N, df)
        
        for docName, tf in tfList.items():
            TWM[term][docName] = tf * idf
            DVL[docName] += TWM[term][docName] ** 2
    
    return TWM, DVL

In [64]:
TWM, DVL = invertedWeight(TDM)

In [73]:
query = "국방의 의무와 보편적 교육에 대한 법안 법안"

queryRepresentation = defaultdict(float)

for token in query.split():
    for morpheme in ma(token):
        queryRepresentation[morpheme] += 1 
        
maxFreq = max(queryRepresentation.values())        
        
for term, freq in queryRepresentation.items():

    df = len(TWM[term])    
    
    if df > 0:
        idf = basicIDF(N, df)
        queryRepresentation[term] = doubleNormalTF(0.5, freq, maxFreq) * idf
    else:
        queryRepresentation[term] = 0

In [74]:
queryRepresentation

defaultdict(float,
            {'국방': 0.5242275032520142,
             '의': 0.0,
             '의무': 0.5242275032520142,
             '와': 0.03431811792050636,
             '보편적': 0,
             '교육': 0.07268250975604232,
             '에': 0.0,
             '대하': 0.0,
             'ㄴ': 0.0,
             '법안': 0.3979400086720376})

In [76]:
candidateList = defaultdict(float)

for term, weight in queryRepresentation.items():
    for docName, docWeight  in TWM[term].items():
        candidateList[docName] += weight * docWeight
        
for docName, similarity in candidateList.items():
    candidateList[docName] = similarity /DVL[docName]

In [79]:
from nltk.tokenize import sent_tokenize

resultList = sorted(candidateList.items(), key= lambda x:x[1], reverse=True)

print("Query: ", query)

for i, (docName, similarity) in enumerate(resultList):
    print("순위:{0} / 문서:{1} / 유사도:{2}".format(
        (i+1), docName, similarity
    ))
    
    content = kobill.open(docName).read()
    print(sent_tokenize(content)[:5])
    print()

Query:  국방의 의무와 보편적 교육에 대한 법안 법안
순위:1 / 문서:1809892.txt / 유사도:0.02336253798221921
['교육공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9892\n\n발의연월일 : 2010.', '11.', '12.', '발  의  자 : 정의화․이명수․김을동 \n\n이사철․여상규․안규백\n\n황영철․박영아․김정훈\n\n김학송 의원(10인)\n\n제안이유 및 주요내용\n\n  현행 교육공무원의 육아휴직은 만 6세 이하의 초등학교 취학 전 자\n\n녀의 육아를 위한 경우로 한정되어 있어 초등학교 취학 후 등하교 및 \n\n방과 후 양육 등에 어려움이 많고, 저학년 자녀 혼자 등하교를 하거나 \n\n어른 없는 집에서 지내다가 성폭력 범죄 등 흉악범죄의 피해자가 되\n\n고 있음.', '이에 육아휴직 가능 시기를 만 8세 이하의 자녀로 확대하여 자녀가 \n\n초등학교에 입학하여 학교생활에 순조롭게 적응할 수 있는 나이까지 \n\n교육공무원이 자녀양육을 위해서 휴직할 수 있도록 하려는 것임(안 제\n\n44조제1항제7호).']

순위:2 / 문서:1809890.txt / 유사도:0.0174873431158863
['지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.', '11.', '12.', '발  의  자 : 정의화․이명수․김을동 \n\n이사철․여상규․안규백\n\n황영철․박영아․김정훈\n\n김학송 의원(10인)\n\n제안이유 및 주요내용\n\n  초등학교 저학년의 경우에도 부모의 따뜻한 사랑과 보살핌이 필요\n\n한 나이이나, 현재 공무원이 자녀를 양육하기 위하여 육아휴직을 할 \n\n수 있는 자녀의 나이는 만 6세 이하로 되어 있어 초등학교 저학년인 \n\n자녀를 돌보기 위해서는 해당 부모님은 일자리를 그만 두어야 하고 \n\n이는 곧 출산의욕을 저하시키는 문제로 이어질 수 있을 것임.', '따라서 육아휴